# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [48]:
Weather_csv_path = "../WeatherPy/City_weather_data.csv"
weather_df = pd.read_csv(Weather_csv_path)
weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hobart,-42.8794,147.3294,62.46,50,20,17.27,AU,1650428332
1,1,Epe,6.5841,3.9834,77.81,91,92,1.77,NG,1650428332
2,2,Lookan,5.0370,120.1027,82.74,75,98,4.99,PH,1650428332
3,3,Chitungwiza,-18.0127,31.0756,53.74,95,100,6.44,ZW,1650428332
4,4,Rawson,-43.3002,-65.1023,61.36,40,99,5.41,AR,1650427962


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [49]:
gmaps.configure(api_key=g_key)
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)



In [50]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=140,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
#A max temperature lower than 27 degrees but higher than 21.

filterdf1 = weather_df[weather_df['Max Temp'] >= 70]
filterdf2 = filterdf1[filterdf1['Max Temp'] <= 80]
filterdf2.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Epe,6.5841,3.9834,77.81,91,92,1.77,NG,1650428332
5,5,Kailua,21.4022,-157.7394,78.78,74,30,7.00,US,1650428307
7,7,Rikitea,-23.1203,-134.9692,78.13,74,15,14.43,PF,1650428333
9,9,Port Macquarie,-31.4333,152.9167,76.89,47,0,10.67,AU,1650428333
12,12,Cockburn Town,21.4612,-71.1419,79.00,80,75,12.19,TC,1650428333


In [31]:
#Wind speed less than 15 kph.
filterdf3 = filterdf2[filterdf2['Wind Speed'] <= 15]
filterdf3.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,Epe,6.5841,3.9834,77.81,91,92,1.77,NG,1650428332
5,5,Kailua,21.4022,-157.7394,78.78,74,30,7.00,US,1650428307
7,7,Rikitea,-23.1203,-134.9692,78.13,74,15,14.43,PF,1650428333
9,9,Port Macquarie,-31.4333,152.9167,76.89,47,0,10.67,AU,1650428333
12,12,Cockburn Town,21.4612,-71.1419,79.00,80,75,12.19,TC,1650428333


In [32]:
#Zero cloudiness.
filterdf4 = filterdf3[filterdf3['Cloudiness'] == 0 ]
filterdf4.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,Port Macquarie,-31.4333,152.9167,76.89,47,0,10.67,AU,1650428333
14,14,Arraial do Cabo,-22.9661,-42.0278,72.34,81,0,2.89,BR,1650428334
91,91,Abu Dhabi,24.4667,54.3667,73.96,82,0,5.75,AE,1650428326
154,154,Lüderitz,-26.6481,15.1594,75.16,27,0,13.04,NaN,1650428488
200,200,Cabo San Lucas,22.8909,-109.9124,74.50,50,0,11.07,MX,1650428495


In [34]:
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
filterdf4.count()
filtered_final = filterdf4[filterdf4['Country'].notna()]
filtered_final.count()

Unnamed: 0    8
City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
Date          8
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = pd.DataFrame(filtered_final, columns= ["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,Port Macquarie,AU,-31.4333,152.9167,
14,Arraial do Cabo,BR,-22.9661,-42.0278,
91,Abu Dhabi,AE,24.4667,54.3667,
200,Cabo San Lucas,MX,22.8909,-109.9124,
331,Vao,NC,-22.6667,167.4833,
346,Najrān,SA,17.4924,44.1277,
486,San Miguel de Cozumel,MX,20.5083,-86.9458,
500,Betioky,MG,-23.7167,44.3833,


In [38]:
target_coordinates = f"{hotel_df['Lat']},{hotel_df['Lng']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [39]:
hotel_name = []


params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    
    name_address = name_address.json()
    
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [41]:
hotel_df['Hotel Name']= hotel_name
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,Port Macquarie,AU,-31.4333,152.9167,Macquarie Waters Boutique Apartment Hotel
14,Arraial do Cabo,BR,-22.9661,-42.0278,Méditerranée Hotel Arraial do Cabo
91,Abu Dhabi,AE,24.4667,54.3667,Grand Hyatt Abu Dhabi Hotel & Residences Emira...
200,Cabo San Lucas,MX,22.8909,-109.9124,Medano Hotel and Suites
331,Vao,NC,-22.6667,167.4833,OURE TERA beach resort
346,Najrān,SA,17.4924,44.1277,NaN
486,San Miguel de Cozumel,MX,20.5083,-86.9458,Wyndham Cozumel Hotel & Resort
500,Betioky,MG,-23.7167,44.3833,NaN


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))